In [164]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import math
from spotipy.exceptions import SpotifyException

# show all columns
pd.set_option('display.max_columns', None)

Audrey's keys: 
* id: 15eeb8c96d3c4e38991a827042884398
* secret: a202e49dc9274a469c7d222428966c1d

Moritz' Keys:
* id: 83578afe5b1d4137a5e813cac12a370a
* secret: 3c4427427adb4a269d0c207b51f3c68d

moritz2:
* id: d8c6553d1c7b4b9781e1ad706e817ad6
* secret: 2b05ae34bbff4e39b7939c09c496075c


In [165]:
# Set up of Spotify API credentials
client_id = "d8c6553d1c7b4b9781e1ad706e817ad6" #Moritz
client_secret = "2b05ae34bbff4e39b7939c09c496075c" #Moritz

# Setting input and output file names
input_file = "billboard_hot_100_2020_2023.csv"
output_file = "billboard_hot_100_audio_features.csv"

# Setting up the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Reading the songs from the input CSV
songs_df = pd.read_csv("Spotify_TT.csv")

In [166]:
# Removing duplicates based on artist name
songs_df = songs_df.drop_duplicates(subset=['artist'], keep='first')

In [167]:
# Erstellen einer Spalte für die Genre
songs_df['Genre API'] = ""

In [168]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13624 entries, 0 to 47544
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          13624 non-null  object 
 1   time_signature    11904 non-null  float64
 2   chorus_hit        11904 non-null  float64
 3   sections          11904 non-null  float64
 4   target            11904 non-null  float64
 5   popularity        1918 non-null   float64
 6   genre             1918 non-null   object 
 7   track             13624 non-null  object 
 8   artist            13624 non-null  object 
 9   duration_ms       13624 non-null  float64
 10  danceability      13624 non-null  float64
 11  energy            13624 non-null  float64
 12  key               13624 non-null  object 
 13  loudness          13624 non-null  float64
 14  mode              13624 non-null  object 
 15  speechiness       13624 non-null  float64
 16  acousticness      13624 non-null  float6

In [169]:
def get_genre_from_uri(uri):
    print("Getting track information...")
    try:
        track = sp.track(uri)
    except SpotifyException as e:
        if e.http_status == 429:
            retry_after = int(e.headers.get('Retry-After', 1))
            print(f"Rate limit exceeded. Waiting for {retry_after} seconds.")
            time.sleep(retry_after)
            return get_genre_from_uri(uri)
        else:
            print(f"Error while getting track information: {e}")
            return ""
    
    artist_id = track["artists"][0]["id"]
    
    print("Getting artist information...")
    artist = sp.artist(artist_id)
    print("Artist information received.")
    
    genres = artist["genres"]
    
    if genres:
        genre_str = ", ".join(genres)
        print(f"Added genres for track URI {uri}: {genre_str}")
        return genre_str
    else:
        print(f"No genres found for track URI {uri}")
        return ""

# Add genres as a new column to your dataframe
def add_genres_with_rate_limit(df, batch_size=1500, start_batch=7):
    num_batches = math.ceil(len(df) / batch_size)

    def process_batch(batch_df):
        for index, row in batch_df.iterrows():
            uri = row['track_id']
            genre_str = get_genre_from_uri(uri)
            print(f"Adding genre '{genre_str}' to DataFrame at index {index}.")
            batch_df.at[index, 'Genre API'] = genre_str
            time.sleep(1 / 3)  # 333 ms
        return batch_df


    for i in range(start_batch, num_batches):
        print(f"Processing batch {i + 1} out of {num_batches}")
        start_index = i * batch_size
        end_index = start_index + batch_size
        df.iloc[start_index:end_index] = process_batch(df.iloc[start_index:end_index])
        
        # Save the processed batch to a CSV file
        df.iloc[start_index:end_index].to_csv(f"V2_batch_{i + 1}.csv", index=False)

add_genres_with_rate_limit(songs_df)


Processing batch 8 out of 10
Getting track information...


KeyboardInterrupt: 

In the next step, we need to re-combine the different batches.

In [ ]:
import glob

def combine_batches_to_single_file(num_batches, output_filename):
    # Use a list to store each batch DataFrame
    df_list = []

    # Loop through the batches
    for i in range(num_batches):
        # Load the batch into a DataFrame
        df_batch = pd.read_csv(f"batch_{i + 1}.csv")
        
        # Append the batch DataFrame to the list
        df_list.append(df_batch)

    # Concatenate all batch DataFrames
    df_combined = pd.concat(df_list, ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    df_combined.to_csv(output_filename, index=False)

# Get the number of batch files
num_batches = len(glob.glob('batch_*.csv'))

# Combine the batch files into a single file
combine_batches_to_single_file(num_batches, "combined.csv")

combined = pd.read_csv("combined.csv")


In [ ]:
batch_8 = pd.read_csv("batch_8.csv")

batch_8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1404 non-null   int64  
 1   track               1404 non-null   object 
 2   artist              1404 non-null   object 
 3   track_id            1404 non-null   object 
 4   danceability_x      1404 non-null   float64
 5   energy_x            1404 non-null   float64
 6   key_x               1404 non-null   object 
 7   loudness_x          1404 non-null   float64
 8   mode_x              1404 non-null   object 
 9   speechiness_x       1404 non-null   float64
 10  acousticness_x      1404 non-null   float64
 11  instrumentalness_x  1404 non-null   float64
 12  liveness_x          1404 non-null   float64
 13  valence_x           1404 non-null   float64
 14  tempo_x             1404 non-null   float64
 15  duration_ms         1404 non-null   int64  
 16  time_s

In [ ]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11904 entries, 0 to 11903
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          11904 non-null  int64  
 1   track               11904 non-null  object 
 2   artist              11904 non-null  object 
 3   track_id            11904 non-null  object 
 4   danceability_x      11904 non-null  float64
 5   energy_x            11904 non-null  float64
 6   key_x               11904 non-null  object 
 7   loudness_x          11904 non-null  float64
 8   mode_x              11904 non-null  object 
 9   speechiness_x       11904 non-null  float64
 10  acousticness_x      11904 non-null  float64
 11  instrumentalness_x  11904 non-null  float64
 12  liveness_x          11904 non-null  float64
 13  valence_x           11904 non-null  float64
 14  tempo_x             11904 non-null  float64
 15  duration_ms         11904 non-null  int64  
 16  time

Find out which genres appear the most in the dataset, to select them and build a dictionary to group them.

In [ ]:
# Split the Genre string into a list of genres
songs_df['Genre_List'] = songs_df['Genre'].apply(lambda x: x.split(', '))

# Expand the list of genres into a series
genre_series = songs_df['Genre_List'].explode()

# Count the number of occurrences of each genre
genre_counts = genre_series.value_counts()

# Convert the genre counts into a DataFrame
genre_counts_df = genre_counts.reset_index()
genre_counts_df.columns = ['Genre', 'Count']


KeyError: 'Genre'

In [ ]:
# Add genres as a new column to your dataframe
songs_df['Genre'] = songs_df['track_id'].apply(get_genre_from_uri)

Getting track information...


KeyboardInterrupt: 

In the next step we match, the artists and genre that we extracted with the full dataset